In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import os
os.getcwd()


'/home/dazai/Documents/Process mining/ProcessMining'

# Define paths, joining traing test and drop unnecessary columns 

In [2]:
path = r"/home/dazai/Documents/Process mining/Data/printer data/"
df_train = pd.read_csv(f'{path}/event-log-training.csv')
df_test = pd.read_csv(f'{path}/event-log-test.csv')


df_data = pd.concat([df_train, df_test])
df_data['event time:timestamp'] = pd.to_datetime(df_data['event time:timestamp'], dayfirst=True)
df_data.sort_values(by=['event time:timestamp'])
df_data.reset_index(inplace=True, drop=True)
df_data.drop(['case description', 'event org:resource'], axis=1, inplace=True)

# remove whitespace at beginning and end of column name
df_data.columns = df_data.columns.str.strip()


# splitting the data using self defined ratios

In [3]:
def split(df_data, percent):
    """input percentage of testing data to split"""
    index = round(df_data.shape[0]*(percent/100))
    
    testset = df_data.iloc[-index:]
    testset.reset_index(drop=True, inplace=True)
    return df_data.iloc[:-index], testset


## Frequency of all events

In [4]:
# events = df_train['event concept:name'].value_counts()
# events.to_frame()

In [5]:
# fig, ax = plt.subplots()

# events = df_train['event concept:name'].value_counts()
# events.plot.barh(figsize=(15,20))

# calculate event time

In [6]:
case = df_data[df_data['case concept:name']==1]
unique_events = case['event concept:name'].unique().tolist()

start = [i for i in unique_events if 'Start' in i]
stop = [i for i in unique_events if 'Stop' in i]

events = start + stop

events = [ele for ele in unique_events if ele not in events]
case

,eventID,case concept:name,case Class,event concept:name,event lifecycle:transition,event time:timestamp
0,0,1,Print,Job,start,1970-01-01 01:00:00
1,1,1,Print,Remote Print,complete,1970-01-01 01:15:00
2,2,1,Print,Read Print Options,complete,1970-01-01 01:26:00
3,3,1,Print,Rasterization,start,1970-01-01 01:38:00
4,4,1,Print,Interpretation,start,1970-01-01 01:51:00
...,...,...,...,...,...,...
851,773,1,Print,Heated Roller Spin Stop,complete,1970-01-07 13:18:00
852,774,1,Print,Fusing,complete,1970-01-07 13:28:00
853,775,1,Print,Wipe Toner on Drum,complete,1970-01-07 13:38:00
854,776,1,Print,Erase Charge on Drum,complete,1970-01-07 14:03:00


# notes
Some events can run in parallel for example, when 1 Job is running another Job can already be initiated. <br><br>
Here the 'next event' is defined as when 1 event is completed, the next event with the same name is regarded as the next event <br><br>
Some events have a naming scheme like 'event-start' and 'event-end'. These will also be calculated in the same way as mentioned above. (not implemented yet
)

In [63]:
def AvgTime(df_data):
    time_diff = {}
    # there are 99 cases classified as either print or copy/scan
    cases = df_data['case concept:name'].unique()

    # Loop through all cases
    for i in cases:
        case = df_data[df_data['case concept:name']==i]
        # Loop through all unique events in one case
        for j in case['event concept:name'].unique():
        
            activity = case[case['event concept:name']== j]
            # remove first line to calculate time between events
            activity.drop(activity.iloc[0].name, inplace=True)

            # Only continues when there are more than 1 event in a case
            if len(activity) > 1:
                # calculates time difference between current and next event
                time  = (activity['event time:timestamp'].shift(-1)-activity['event time:timestamp'])
                if j not in time_diff.keys():
                    time_diff[j] = time.mean()
                else:
                    time_diff[j] = time_diff[j] + time.mean()/2

    return pd.DataFrame.from_dict(time_diff, orient='index', columns=['Average Time'])['Average Time'].dt.round('s').to_frame().sort_values(by=['Average Time'])


In [64]:
AvgTime(df_data)

,Average Time
Unformatted Text,0 days 05:07:11
Page Control Language,0 days 06:32:08
Post Script,0 days 07:35:27
Interpretation,0 days 09:53:51
Screening,0 days 16:18:55
Error Diffusion Method,0 days 20:59:01
Coat Light Toner on Drum,0 days 21:30:08
Load Quantizing Pixel,0 days 22:08:02
Neighbor Quant Error Packingl,0 days 22:10:02
Subtract,0 days 22:14:00
